In [1]:
from transformers import AutoTokenizer
import transformers
import torch
import accelerate
from tqdm import tqdm
print(transformers.__version__)

C:\ProgramData\Anaconda3\envs\torchgpu\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


4.33.0


{'error': 'Model requires a Pro subscription; check out hf.co/pricing to learn more. Make sure to include your HF token in your query.'}


In [2]:
import os
import json
def read_jsons(file_path):
    '''
    all_contents: a dict contains {title:content},
    content: a dict contains {"page_x_text":text}
    '''
    all_contents = {}
    for file in os.listdir(file_path):
        if file.endswith(".json"):
            f = open(file_path+"/"+file)
            passage = json.load(f)
            article = passage['Article']
            text_label = 'plum_contents'
            if text_label not in passage:
                text_label = 'ocr_contents'
            contents = passage[text_label]
            f.close()

            all_contents[article] = contents

    return all_contents
file_path="json"
all_contents = read_jsons(file_path)

In [3]:
meta_data = []
for i in all_contents.keys():
    article = i
    article_text =  all_contents[article]
    article_text = list(article_text.values())
    for pages in range(len(article_text)):
        page_text = article_text[pages]
        sentences = page_text.split(".")
        for j in range(len(sentences)):
            sentence = sentences[j]
            if len(sentence)> 50:
                dic = {}
                dic["text"] = sentences[j]
                meta_data.append(dic)

In [4]:
print(meta_data[10:20])

[{'text': ' These areas are arguably strong fits for blockchain'}, {'text': ' These industrial use cases are believed to deliver real ROI at an early stage of blockchain development (Biinger, 2017)'}, {'text': '  Among many activities that are likely to be transformed by block- chain, supply chain thus deserves special attention'}, {'text': ' An increasing re- liance on the use of Internet-of-things (IoT) applications is among the trends that will affect supply chain management (SCM)'}, {'text': ' With IoT, radio-frequency identification (RFID) tags, sensors, barcodes, GPS tags and chips, the locations of products, packages and shipping containers can be tracked at each step'}, {'text': ' This allows an enhanced, real-time tracking of goods from their origins'}, {'text': '  In this regard, for one thing, there is a deep thirst for a foolproof method for confirmed identity in IoT applications'}, {'text': " The first of block- chain's direct benefits is that it provides a possible soluti

In [5]:
category = '''
Category 1: General Actions
Produce, create, present, implement, construct, operate, perform, conduct, run, tend, carry, start, build

Category 2: Communication and Expression
describe, express, indicate, note, suggest, propose, discuss, explain, report, represent, illustrate, refer, mention, reveal, specify

Category 3: Connection and Interaction
connect, relate, associate, link, integrate, combine, incorporate, meet, interact, engage, cooperate, communicate, collaborate

Category 4: Supplies
give, share, offer, provide, supply, deliver

Category 5: Requirement and Demand
need, require, demand, depend

Category 6: Acquisition and Exchange
order, obtain, purchase, buy, acquire, receive, trade, sell

Category 7: Change and Transformation
change, transform, adapt, shift

Category 8: Decision and Choice
choose, decide, select, pick, opt

Category 9: Measurement and Assessment
measure, assess, evaluate, calculate, test, examine, investigate

Category 10: Problem Solving
solve, fix, resolve, address

Category 11: Growth and Development
increase, develop, improve, grow, progress, advance, evolve, proceed

Category 12: Influence and Impact
affect, influence, Impact, contribute, facilitate

Category 13: Control and Management
manage, control, coordinate, regulate, arrange, organize, sort

Category 14: Motion
move, go, travel, journey, walk

Category 15: Time
occur, happen, continue, remain
'''

In [3]:
model = "meta-llama/Llama-2-7b-chat-hf"
device="cuda"
tokenizer = AutoTokenizer.from_pretrained(model)

In [4]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    #torch_dtype=torch.float16,
    torch_dtype=torch.float32,
    device_map="auto",
    #device="cuda" #cuda
)

sequences = pipeline(
    'I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?\n',
    eos_token_id=tokenizer.eos_token_id,
    max_length=200,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████| 2/2 [00:25<00:00, 12.92s/it]


Result: I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?

I'm open to a wide range of genres, but I tend to prefer shows with complex characters and storylines.

Thanks!


In [18]:
file_name = "output.txt"


for text_dict in tqdm(meta_data[10:11]):


    prompt = f'''
RDF stands for Resource Description Framework, and it is a widely used format for representing information in the form of subject-predicate-object triplets in Knowledge Graph. Each RDF triplet consists of three components:

Subject: The subject is typically a resource or entity that you want to describe.
Predicate: The predicate, also known as the property or relationship, describes the relationship between the subject and the object.
Object: The object is the value or target of the statement.

Suppose you are an RDF triplet extraction model. I will give you 15 categories of predicates in the field of supply chain.

{category}

Give you some sentences, please extract all the RDF triplets based on the categories. Please form each triplet as (subject, predicate, object) + (predicate category) in the output. If no triplet is found, answer: None. Do not include other words in the answer.

Sentence: {text_dict['text']}

Triplets:'''


    output = pipeline(prompt,
    eos_token_id=tokenizer.eos_token_id
                     )
    output_text = output[0]['generated_text']
    output_text = output_text.replace(prompt, '')

    with open(file_name, "a") as file:
        file.write(output_text + '\n')

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 172.00 MiB (GPU 0; 6.00 GiB total capacity; 5.05 GiB already allocated; 0 bytes free; 5.21 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF